# Chapter 4: Acceralation of word2vec

The problem of word2vec which implemented previous chapter is that it would be slow if the courpus is huge. So, we need to improve word2vec.

## 4.1 word2vec improvement : Embedded class



In [5]:
import numpy as np

W = np.arange(21).reshape(7, 3)

print(W)
print(W[2])

idx = np.array([1, 0, 2])
print(W[idx])

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]
[6 7 8]
[[3 4 5]
 [0 1 2]
 [6 7 8]]


In [6]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        # np.add.at(dW, self.idx, dout)
        # return None
        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]
        return None

## 4.2 word2vec improvement : Negative sampling

### 4.2.1 calculation problem after middle layer

When the corpus size is huge, calculation takes time in following process.

- the product of the neurons of the hidden layer and the weight matrix
- calculation of Softmax layer

For example, softmax is shown as following assuming that the corpus size is 100,000.

$$
y = \frac{\exp(x_i)}{\sum_j^{1000000} \exp(x_j)}
$$

### 4.2.2 Negative sampling

To reduce the calculation problem, we use negative sampling in which we randomly select negative samples from the corpus.

### 4.2.3 Sigmoid function and Cross entropy error

ToDo...

### 4.2.6 Sampling method of negative sampling